In [1]:
import queue
import time
import os
import sys

from DataHandler.LOBHourlyDataHandler import HistoricLOBHourlyDataHandler
from Strategy.strategy import BuyAndHoldStrategy
from Portfolio.LogPlotPortfolio import LogPlotPortfolio
from Execution.execution import SimulatedExecutionHandler

event_queue = queue.Queue()
data_handler = HistoricLOBHourlyDataHandler(event_queue, 
                                           symbol_list=['btc_usdt','eth_usdt','xrp_usdt'],
                                           exchange_list=['binance','binance','binance'], 
                                           file_dir = 'data_sample/small_sample/', 
                                           # file_dir = 'data_sample/',
                                           is_csv=False)
#sys.exit()

portfolio = LogPlotPortfolio(event_queue, data_handler)      # 组合
executor = SimulatedExecutionHandler(event_queue, data_handler)   # 回测模拟成交器；如果是实盘这里就是算法交易模块
strategy = BuyAndHoldStrategy(event_queue, data_handler, portfolio, executor)       # 策略实例。实际应用中应该有多个策略实例

cnt = 0
while True:
    # Update the trade/LOB (specific backtest code, as opposed to live trading)
    if data_handler.continue_backtest == True:
        data_handler.update_TradeLOB()
    else:
        break
    
    cnt+=1
    if cnt>2000: break

    # Handle the events
    while True:
        try:
            event = event_queue.get(False)
        except queue.Empty:
            break
        else:
            if event is not None:

                if event.type == 'MARKET':
                    print('get market event', event)
                    strategy.calculate_signals(event)
                    portfolio.update_holdings_from_market()
                    executor.on_market_event(event)

                elif event.type == 'ORDER':
                    executor.on_order_event(event)

                elif event.type == 'FILL':
                    print('get fill event', event)
                    portfolio.update_positions_from_fill(event)
                    strategy.on_order_fill(event)
                    #sys.exit()

backtest on:  ['btc_usdt_binance', 'eth_usdt_binance', 'xrp_usdt_binance']
/*----- start initialize the DataHandler -----*/
/*----- DataHandler initialization ends -----*/

===== reload data from new hour =====

===== processing market event in  1702746000000  =====
get new market events and push to queue
get market event {'type': 'MARKET'}

===== processing market event in  1702746000001  =====
get new market events and push to queue
get market event {'type': 'MARKET'}
put orders to the exexution {'type': 'ORDER', 'timestamp': 1702746000101, 'symbol': 'btc_usdt_binance', 'order_id': 0, 'order_type': 'MARKET', 'direction': 'BUY', 'quantity': 0.02347961288692641, 'price': None}
put orders to the exexution {'type': 'ORDER', 'timestamp': 1702746000101, 'symbol': 'eth_usdt_binance', 'order_id': 1, 'order_type': 'MARKET', 'direction': 'BUY', 'quantity': 0.44382703172919447, 'price': None}

===== processing market event in  1702746000002  =====
get new market events and push to queue
get mar

In [2]:
portfolio.all_holdings

{'btc_usdt_binance': {1702746000154: 1000.0,
  1702746000171: 999.9998826019355,
  1702746004591: 999.9767551832418,
  1702746004593: 999.9654849690562,
  1702746004594: 999.9498710264866,
  1702746004603: 999.9486970458421,
  1702746004624: 999.923808656182,
  1702746004628: 999.9026770045838,
  1702746004634: 999.9029118007127,
  1702746004640: 999.905729354259,
  1702746004641: 999.9080773155476,
  1702746004651: 999.9029118007127,
  1702746004660: 999.9026770045838,
  1702746004664: 999.9024422084549,
  1702746004678: 999.8759102458926,
  1702746004681: 999.8737970807327,
  1702746004690: 999.8694533523487,
  1702746004701: 999.8688663620266,
  1702746004712: 999.8679271775111,
  1702746004719: 999.8658140123513,
  1702746004729: 999.8624094684826,
  1702746004730: 999.8559525749388,
  1702746004748: 999.8550133904232,
  1702746004750: 999.8540742059079,
  1702746005355: 999.8507870601037,
  1702746005422: 999.849847875588,
  1702746006255: 999.8356427097915,
  1702746006256: 999.8